# Como máquinas aprendem: Gradiente Descendente

"*Um programa de computador é dito que **aprende** de uma experiência E em respeito a uma série de tarefas T e medido com uma performance P, se P em T melhora com as experiências E*" <br> Tom Mitchell.
Já construímos a noção de que o **ato de um algoritmo aprender uma tarefa é o ato de descoberta e reprodução da função interna (mapping) do nosso dataset**. Para descobrir essa função, o algoritmo ajustará seus parâmetros (otimização)

# MQO - Forma Algébrica

$Y = Xβ + e$

$β = (X^T X)^{-1} X^T Y$

Onde:
- β é o **vetor** de pesos (nossos parâmetros estimados)
- X é a matriz de valores, onde cada linha é uma observação e cada coluna é uma variável
- o Sobrescrito T indica "Transposto", ou seja, a matriz com um T é a matriz transposta.
- y é simplesmente o vetor da variável y.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [6]:
data = pd.read_csv(r'train.csv')

In [7]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [8]:
x = data['GrLivArea']
y = data['SalePrice']

In [9]:
x.head()

0    1710
1    1262
2    1786
3    1717
4    2198
Name: GrLivArea, dtype: int64

In [11]:
x = (x - x.mean()) / x.std() # normalizando
x.head()

0    0.370207
1   -0.482347
2    0.514836
3    0.383528
4    1.298881
Name: GrLivArea, dtype: float64

In [12]:
x.shape

(1460,)

In [18]:
x.head(5)

0    0.370207
1   -0.482347
2    0.514836
3    0.383528
4    1.298881
Name: GrLivArea, dtype: float64

In [13]:
temp = np.c_[np.ones(1460), x] # colocando uma "coluna de 1s" (será nosso $\beta_0$)

In [16]:
temp

array([[ 1.        ,  0.37020659],
       [ 1.        , -0.48234664],
       [ 1.        ,  0.51483616],
       ...,
       [ 1.        ,  1.56910958],
       [ 1.        , -0.83250243],
       [ 1.        , -0.49376477]])

In [22]:
x = np.c_[np.ones(x.shape[0]), x] # colocando uma "coluna de 1s" (será nosso $\beta_0$)
x

array([[ 1.        ,  0.37020659],
       [ 1.        , -0.48234664],
       [ 1.        ,  0.51483616],
       ...,
       [ 1.        ,  1.56910958],
       [ 1.        , -0.83250243],
       [ 1.        , -0.49376477]])

In [26]:
w = np.zeros(x.shape[1])

z = np.linalg.inv(np.dot(x.T, x))

w = np.dot(z, np.dot(x.T, y))

In [27]:
w

array([180921.19589041,  56294.90210563])

In [28]:
z

array([[6.84931507e-04, 9.63172444e-20],
       [9.63172444e-20, 6.85400960e-04]])

In [29]:
b, w1 = w[0], w[1]

print('Inclinação: %.2f' % w1)
print('Intercepto: %.2f' % b)

Inclinação: 56294.90
Intercepto: 180921.20


In [32]:
# Vamos checar com nosso statsmodel
import statsmodels.formula.api as smf

data['GrLivArea_norm'] = (data['GrLivArea']-data['GrLivArea'].mean())/data['GrLivArea'].std()

def run_regression(df,function):
    model = smf.ols(function, df).fit()
    print(model.summary())
run_regression(data,'''SalePrice ~ GrLivArea_norm''')

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.502
Model:                            OLS   Adj. R-squared:                  0.502
Method:                 Least Squares   F-statistic:                     1471.
Date:                Wed, 15 May 2019   Prob (F-statistic):          4.52e-223
Time:                        10:37:55   Log-Likelihood:                -18035.
No. Observations:                1460   AIC:                         3.607e+04
Df Residuals:                    1458   BIC:                         3.608e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       1.809e+05   1467.490    123.